In [1]:
from keras.layers import *
from keras.models import Model
from keras.activations import relu

import numpy as np
import tensorflow as tf
import keras.backend as K

Using TensorFlow backend.
/data/home/chenzhiyuan/anaconda3/envs/tf/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/data/home/chenzhiyuan/anaconda3/envs/tf/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/data/home/chenzhiyuan/anaconda3/envs/tf/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/d

In [2]:
# 解决keras 显存问题
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.25
set_session(tf.Session(config=config)) 

In [3]:
input_length = 10
input_dim = 1
MAX_LENGTH = 300 # 单个字符的长度
output_length = 10
output_dim = 6

samples = 100
hidden_dim = 6
hidden_size = 300 # a half of Max_LENGTH

In [4]:
#embedding_size = [embedding_length,embedding_dim,input_length]
embedding_size = [1000,64,300]
# hidden_size is the length of the kernel utils  = enc_units
class EncoderRNN(Model):
    def __init__(self, embedding_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = Embedding(embedding_size[0], embedding_size[1],input_length=embedding_size[2])
        self.gru = GRU(hidden_size, return_sequences=True, return_state=True)
        self.deepgru = GRU(hidden_size*2, return_sequences=True, return_state=True)
        self.lastdeepgru = GRU(hidden_size*2, return_sequences=False, return_state=True)
        self.bigru = Bidirectional(self.gru)

    def __call__(self, en_input):
        state_h = []
        emb = self.embedding(en_inputs)
        encoder_out, fwd_h1, bck_h1 = self.bigru(emb)
        state_h.append(concatenate([fwd_h1, bck_h1]))
        if hidden_dim>1:
            for i in range(1,hidden_dim):
                encoder_out, en_hidden = self.deepgru(encoder_out)
                state_h.append(en_hidden)
        output = encoder_out
        hidden = state_h
        return output, hidden
    
    def initialize_hidden_state(self):
        init_state = [tf.zeros((1, self.hidden_size)) for i in range(2)]
        return init_state

In [5]:
class DecoderRNN(Model):
    def __init__(self, embedding_size, hidden_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = Embedding(embedding_size[0], embedding_size[1],input_length=embedding_size[2])
        self.gru = GRU(hidden_size, return_sequences=True, return_state=True)
        self.deepgru = GRU(hidden_size*2, return_sequences=True, return_state=True)
        self.lastdeepgru = GRU(hidden_size*2, return_sequences=False, return_state=True)
        self.bigru = Bidirectional(self.gru)
        
        self.out = Dense(output_length)
        self.softmax = Activation('softmax')

    def __call__(self, de_input, hiddens):
        state_h = []
        emb = self.embedding(en_inputs)
        emb = Activation('relu')(emb)
        # !挖个坑，这里没有初始化hidden_state
        output, fwd_h, bck_h = self.bigru(emb)
        state_h.append(concatenate([fwd_h, bck_h]))
        if len(hiddens)>2:
            for hidden in hiddens[1:-1]:
                output, de_hidden = self.deepgru(output, hidden)
                state_h.append(de_hidden)
        if len(hiddens)>1:
            output, de_hidden = self.lastdeepgru(output, hidden)
            state_h.append(de_hidden)
        hidden = state_h
        output = self.softmax(output)
        print(type(output))
        return output, hidden

    def initialize_hidden_state(self):
        init_state = tf.zeros((1, self.hidden_size))
        return init_state

In [6]:

class AttnDecoderRNN(Model):
    def __init__(self, embedding_size ,hidden_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = Embedding(embedding_size[0], embedding_size[1],input_length=embedding_size[2],name ='atten_embed')
        self.attn = Dense(self.max_length,name = 'atten_attn')
        self.attn_combine = Dense(self.hidden_size,name = 'atten_combine')
        self.dropout = Dropout(self.dropout_p,name ='atten_dropout')
        
        self.gru = GRU(hidden_size, return_sequences=True, return_state=True,name = 'atten_gru')
        self.deepgru = GRU(hidden_size*2, return_sequences=True, return_state=True,name = 'atten_deepgru')
        self.lastdeepgru = GRU(hidden_size*2, return_sequences=False, return_state=True,name='atten_lastdeepgru')
        self.bigru = Bidirectional(self.gru,name='atten_bigru')
        self.batch_dot = Lambda(lambda layers:K.batch_dot(layers[0],layers[1]))
        self.out = Dense(output_length)
        self.softmax =Softmax(axis=-1)
        #包装层
        
        
        

    def __call__(self, de_input, hiddens, encoder_outputs):
        state_h = []
        embedded = self.embedding(de_input)
        print(embedded.shape)
        # TODO ; use lambda https://www.cnblogs.com/jqpy1994/p/11433746.html or  https://keras.io/zh/layers/core/
        #embedded = K.reshape(embedded,[samples,embedding_size[1]*embedding_size[2]])
        embedded = Reshape((1,embedding_size[1]*embedding_size[2]))(embedded)
        print(type(embedded))
        embedded = Lambda(lambda x:K.squeeze(x,1))(embedded)
    
        embedded = self.dropout(embedded) # Dim:(Batch Size , Decoder Hidden Size + Embedding Size)
        if type(hiddens) == type(list()):
            hidden = hiddens[-1]
        
        # hidden = K.reshape(hidden,[samples,self.hidden_size*2]) 
        hidden = Reshape((1,self.hidden_size*2))(hidden)
        hidden = Lambda(lambda x:K.squeeze(x,1))(hidden)
        concat = Concatenate(1,name='atten_concat2')([embedded, hidden])
        # note: 从这里开始，把两个向量拼接起来    
        attn_weights =self.softmax(
            Dense(self.max_length)(concat))
        atten_weights = Reshape((1,-1))(attn_weights)
        attn_applied = self.batch_dot([atten_weights,encoder_outputs])
        print(attn_applied.shape)
        attn_applied = Lambda(lambda x:K.squeeze(x,1))(attn_applied)

        
        output = Concatenate(1)([embedded, attn_applied])

        output = self.attn_combine(output)
        
        output = ReLU()(output)
        output = Reshape((1,-1))(output)
        output, fwd_h, bck_h = self.bigru(output)
        state_h.append(concatenate([fwd_h, bck_h]))
        if len(hiddens)>2:
            for hidden in hiddens[1:-1]:
                output, de_hidden = self.deepgru(output, hidden)
                state_h.append(de_hidden)
        if len(hiddens)>1:
            output, de_hidden = self.lastdeepgru(output, hidden)
            state_h.append(de_hidden)
        hidden = state_h
#         output = self.softmax(output)
        output = Reshape((1,-1))(output)
        output = TimeDistributed(Dense(MAX_LENGTH, activation='softmax'))(output)
        output = Lambda(lambda x:K.squeeze(x,1))(output)
        print(output.shape)
        return output, hidden
    
    
    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [7]:
import numpy as np
import os
from keras.callbacks import TensorBoard
from keras.models import Model,load_model
#############
# 写表文件
stem_tab_file = '../table/stem.txt'
dbn_tab_file = '../table/dbn.txt'
flag_tab_file = '../table/flag.txt'
#
#############
# 训练文件和测试文件
train_folder = '../data/train/'
test_folder = '../data/test/'
vaild_folder = '../data/vaild/'

dbn_file_in ='dbn.in.txt'
dbn_file_out ='dbn.out.txt'

flag_file_in ='flag.in.txt'
flag_file_out = 'flag.out.txt'
#
#############
# 保存模型
model_file ='./save_model/keras_seq2seq_embedding.h5'
if not os.path.exists('./save_model'):
    os.makedirs('./save_model')

In [8]:
# 解决keras 显存问题
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
set_session(tf.Session(config=config)) 

In [9]:

class Base:
    def __init__(self,filename,num_word_threshold):
        self._bash_to_id = {}
        self._num_word_threshold = num_word_threshold
        self._read_dict(filename)
    
    def _read_dict(self,filename):
        with open(filename,'r') as f:
            lines = f.readlines()
        for line in lines:
            # print(line)
            word,frequency = line.strip('\r\n').split('\t')
            frequency = int(frequency)
            if frequency < self._num_word_threshold:
                continue
            idx = len(self._bash_to_id)
            if word == '<UNK>':
                self._unk = idx
            elif word == '<PAD>':
                self._pad = idx
            elif word == '<GO>':
                self._go = idx
            elif word == '<EOS>':
                self._eos = idx
            self._bash_to_id[word] = idx
            
    @property
    def unk(self):
         return len(self._bash_to_id)
    @property
    def pad(self):
         return len(self._bash_to_id)+1 
    @property
    def go(self):
         return len(self._bash_to_id)+2
    @property
    def eos(self):
         return len(self._bash_to_id)+3
        
    def size(self):
         return len(self._bash_to_id)
    def base_to_id(self,word):
        #完成
         return self._bash_to_id.get(word,self._unk)
    def sequence_to_id(self,sequeuce):
        word_ids = [self.base_to_id(cur_word) for cur_word in sequeuce.split()]
        return word_ids
    
    ### 
    # 不需要id2base 所以没写
    ###
num_word_threshold = 5000 # 频率低于5000的不要
base =Base(stem_tab_file,num_word_threshold)

In [10]:
# base.sequence_to_id("A G C U A C G G C C A U A C A U A G A U G A A A A U A C C G G A U C C C G U C C G A U C U C C G A A G U C A A G C A U C U A A U G G C G A C G U C A G U A C U G U G A U G G G G G A C C G C A C G G G A A U A C G U C G U G C U G U A G U")

In [11]:
class CategoryDict:
    def __init__(self,filename):
        self._category_to_id = {}
        self._id_to_category = {}
        with open(filename , 'r') as f: 
            lines = f.readlines()
        for line in lines:
            category,_ = line.strip('\r\n').split('\t')
            idx = len(self._category_to_id)
            self._category_to_id[category] = idx
            self._id_to_category[idx] = category
    def size(self):
        return len(self._category_to_id)
    
    
    @property
    def go(self):
         return len(self._category_to_id)
    @property
    def eos(self):
         return len(self._category_to_id)+1
    @property
    def pad(self):
         return len(self._category_to_id)+2
        
    def category_to_id(self,category):
        if category not in self._category_to_id:
            raise Exception(
                    "%s is not in our category list" % category)
        return self._category_to_id[category]  
    def id_to_category(self,id):
        if id not in self._id_to_category:
            raise Exception(
                    "%s is not in our id list" % id)
        return self._id_to_category[id]  
    
    def sequence_to_id(self,sequeuce):
        word_ids = [self.category_to_id(cur_word) for cur_word in sequeuce.split()]
        return word_ids
    
    def id_to_sequence(self,ids):
        category = ' '.join([self.id_to_category(_id) for _id in ids])
        return category
    
category_base = CategoryDict(dbn_tab_file)
test_str = '. ( )'
test_str2 = [0,1,2,3]
print(category_base.sequence_to_id(test_str))

[1, 3, 2]


In [12]:
# 构建数据库 对其，shuffle，batch

num_timesteps = 300

class TextDataSet:
    def __init__(self, folder, vocab, category_vocab, num_timesteps,):
        infile = folder + dbn_file_in
        outfile = folder + dbn_file_out
        self._vocab = vocab
        self._category_vocab = category_vocab
        self._num_timesteps = num_timesteps
        # matrix 矩阵
        self._inputs = []# 所有数据的集合
        # vector 列表
        self._targets = []#
        self._deinputs = []
#         self._target_out = []# 
        self._label_len = []
        self._feature_len = []
        self._deinput_len = []
        self._indicator = 0# batch的位置
        self._parse_file(infile,outfile) # 解析⽂件

        
        
    def _parse_file(self,infile,outfile): 
        """ 解析文件 """
        
        print('Loading data from %s',infile)
        print('Loading data from %s',outfile)
        with open(infile,'r') as fi:
            inlines = fi.readlines()
        with open(outfile,'r')as fo:
            outlines = fo.readlines()
            
        for line in inlines:
            line = line.strip('\r\n').strip()
            id_feature = self._vocab.sequence_to_id(line)
            id_feature = id_feature[0:self._num_timesteps] #保证不会过长
            feature_len = len(id_feature)
            self._feature_len.append(feature_len)
            padding_num = self._num_timesteps - feature_len # 如果id_words⼩于这个num_timesteps
            id_feature = [self._vocab.go]+id_feature + list(self._vocab.pad for i in range(padding_num-1))
            self._inputs.append(id_feature)
            
        for line in outlines:
            line = line.strip('\r\n')
            id_label = self._category_vocab.sequence_to_id(line)
            id_label = id_label[0:self._num_timesteps]#600
            label_len = len(id_label)
            self._deinput_len.append(label_len) 
            padding_num = self._num_timesteps - len(id_label) # 如果id_words⼩于这个num_timesteps
            id_label = [self._category_vocab.go]+id_label + list(self._category_vocab.pad for i in range(padding_num-1))
            self._deinputs.append(id_label)
            
        for line in outlines:
            line = line.strip('\r\n')
            id_label = self._category_vocab.sequence_to_id(line)
            id_label = id_label[0:self._num_timesteps]#600
            label_len = len(id_label)
            self._label_len.append(label_len)
            padding_num = self._num_timesteps - len(id_label) # 如果id_words⼩于这个num_timesteps
            id_label = id_label + [self._category_vocab.eos]+ list(self._category_vocab.pad for i in range(padding_num-1))
            self._targets.append(id_label)        
        
            
            
        self._inputs = np.asarray(self._inputs, dtype = np.int32)
        self._target = np.asarray(self._targets, dtype = np.int32)
        self._deinputs = np.asarray(self._deinputs, dtype = np.int32)
#         self._target_out = np.asarray(self._target_out, dtype = np.int32)
        self._label_len = np.asarray(self._label_len, dtype = np.int32)
        self._feature_len = np.asarray(self._feature_len, dtype = np.int32)
        self._random_shuffle()

    def _random_shuffle(self):
        p = np.random.permutation(len(self._inputs))
        self._inputs = self._inputs[p]
        self._target = self._target[p]
        self._label_len = self._label_len[p]
        self._feature_len = self._feature_len[p]

    def next_batch(self,batch_size):
        end_indicator = self._indicator +batch_size
        if end_indicator > len(self._inputs):
            self._random_shuffle()
            self._indicator = 0
            end_indicator = batch_size
        if end_indicator > len(self._inputs):# 则说明batchsize ⽐样本空间还要⼤
            raise Exception("batch_size:%d is too large"% batch_size)
        
        batch_inputs = self._inputs[self._indicator:end_indicator]
        batch_target = self._target[self._indicator:end_indicator]
        batch_deinput = self._deinputs[self._indicator:end_indicator]
        inputs_len = self._feature_len[self._indicator:end_indicator]
        outputs_len =  self._label_len[self._indicator:end_indicator]
        deinputs_len = self._deinput_len[self._indicator:end_indicator]
        
        self._indicator = end_indicator
        return (batch_inputs,inputs_len),(batch_target,outputs_len),(batch_deinput,deinputs_len)
train_dataset = TextDataSet(
    train_folder, base, category_base, num_timesteps)
print('done')
val_dataset = TextDataSet(
    vaild_folder, base, category_base, num_timesteps)
print('done')
test_dataset = TextDataSet(
    test_folder, base, category_base, num_timesteps)
print('done')


Loading data from %s ../data/train/dbn.in.txt
Loading data from %s ../data/train/dbn.out.txt
done
Loading data from %s ../data/vaild/dbn.in.txt
Loading data from %s ../data/vaild/dbn.out.txt
done
Loading data from %s ../data/test/dbn.in.txt
Loading data from %s ../data/test/dbn.out.txt
done


In [13]:
a = train_dataset.next_batch(2)
print(a)
# print(val_dataset.next_batch(2)) 
# print(test_dataset.next_batch(2))

((array([[8, 4, 3, 3, 3, 4, 2, 5, 3, 2, 2, 4, 3, 5, 4, 4, 5, 3, 5, 4, 4, 2,
        5, 4, 2, 2, 2, 3, 4, 4, 4, 5, 2, 3, 4, 4, 4, 5, 3, 4, 5, 3, 5, 5,
        3, 2, 4, 4, 4, 2, 4, 2, 2, 2, 4, 4, 2, 5, 2, 2, 5, 5, 4, 4, 4, 3,
        5, 2, 4, 2, 5, 3, 2, 5, 3, 5, 2, 2, 4, 2, 3, 5, 2, 4, 3, 5, 4, 4,
        5, 3, 3, 3, 3, 2, 4, 3, 5, 5, 3, 3, 3, 3, 5, 5, 2, 3, 4, 4, 3, 3,
        4, 4, 2, 4, 3, 5, 5, 4, 4, 4, 4, 5, 3, 5, 5, 5, 4, 4, 3, 2, 5, 3,
        3, 2, 3, 4, 4, 2, 4, 5, 4, 4, 2, 3, 3, 4, 4, 2, 7, 7, 7, 7, 7, 7,
        7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
        7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
        7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
        7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
        7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
        7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
        7, 7, 7, 7, 7, 7, 7, 7, 7, 7

In [14]:
# 写数据
def write_log(callback, names, logs, batch_no):

    for (name, value) in zip(names, logs):
        summary = tf.Summary()
        summary_value = summary.value.add()
        summary_value.simple_value = value
        summary_value.tag = name
        callback.writer.add_summary(summary, batch_no)
        callback.writer.flush()
        
def write_log_batch(callback, name, log, batch_no):
    value = log
    summary = tf.Summary()
    summary_value = summary.value.add()
    summary_value.simple_value = value
    summary_value.tag = name
    callback.writer.add_summary(summary, batch_no)
    callback.writer.flush()

In [15]:
# input_length = 10
# input_dim = 1
# MAX_LENGTH = 300 # 单个字符的长度
# output_length = 10
# output_dim = 4

# samples = 100
# hidden_dim = 3
# hidden_size = 150 # must be a half of Max_LENGTH

In [16]:
####
# model

encoder = EncoderRNN(embedding_size,hidden_size)
decoder = DecoderRNN(embedding_size,hidden_size)
atten = AttnDecoderRNN(embedding_size,hidden_size)# hidden_size = 6

en_inputs = Input(shape=(300,),dtype=float)
de_inputs = Input(shape=(300,),dtype=float)
en_output, en_hidden = encoder(en_inputs)
attent_output,attn_hidden = atten(de_inputs,en_hidden,en_output)

#
######

W1003 22:28:23.994187 140630717482752 deprecation_wrapper.py:119] From /data/home/chenzhiyuan/anaconda3/envs/tf/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1003 22:28:24.004379 140630717482752 deprecation_wrapper.py:119] From /data/home/chenzhiyuan/anaconda3/envs/tf/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1003 22:28:24.008403 140630717482752 deprecation_wrapper.py:119] From /data/home/chenzhiyuan/anaconda3/envs/tf/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1003 22:28:25.888438 140630717482752 deprecation_wrapper.py:119] From /data/home/chenzhiyuan/anaconda3/envs/tf/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.p

(?, 300, 64)
<class 'tensorflow.python.framework.ops.Tensor'>
(?, 1, 600)
(?, 300)


In [20]:
model = Model([en_inputs,de_inputs], attent_output)
if os.path.exists(model_file):
    print("load weight from ",model_file)
    model.load_weights(model_file)
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
model.summary()

log_path = './graph'
callback = TensorBoard(log_path)
callback.set_model(model)
train_names = 'train_loss'
val_names = 'val_loss'
for batch_no in range(100000):
    #####
    # 处理数据
    X_train_all, Y_train_all,Y_deinput_all = train_dataset.next_batch(samples)
    X_train,length = X_train_all
    Y_train,length = Y_train_all
    Y_deinput,length = Y_deinput_all
    # 处理数据
    #######
#   (samples, input_length, input_dim) 16,300,1
    logs = model.train_on_batch([X_train, Y_deinput],Y_train )

    print("[train]:step:%d,loss:%f,acc:%f "%(batch_no,logs[0],logs[1]))
    write_log_batch(callback, train_names, logs[0], batch_no)
    if batch_no % 10 == 0:
        model.save_weights(model_file)
        #####
        # 处理数据
        X_val_all, Y_val_all , Y_de_all= val_dataset.next_batch(16)
        X_val,length = X_val_all
        Y_val,length = Y_val_all
        Y_de,length = Y_de_all
        # 处理数据
        #######
#         logs = model.test_on_batch(X_val, Y_val)
        score = model.evaluate([X_val,Y_de], Y_val, verbose=0)

        print("[vaild]:step:%d,loss:%f,acc:%f "%(batch_no,score[0],score[1]))
        
        write_log_batch(callback, val_names, logs[0], batch_no//10)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 300)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 300, 64)      64000       input_1[0][0]                    
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) [(None, 300, 600), ( 657000      embedding_1[0][0]                
__________________________________________________________________________________________________
gru_2 (GRU)                     [(None, 300, 600), ( 2161800     bidirectional_1[0][0]            
                                                                 gru_2[0][0]                      
          

[train]:step:18,loss:6831.465820,acc:0.000000 
[train]:step:19,loss:6728.345215,acc:0.000000 
[train]:step:20,loss:6755.117676,acc:0.000000 
[vaild]:step:20,loss:6664.409180,acc:0.000000 
[train]:step:21,loss:6692.052734,acc:0.000000 
[train]:step:22,loss:6509.510742,acc:0.000000 
[train]:step:23,loss:6717.941895,acc:0.000000 
[train]:step:24,loss:6754.718750,acc:0.000000 
[train]:step:25,loss:6684.495117,acc:0.000000 
[train]:step:26,loss:6770.722656,acc:0.000000 
[train]:step:27,loss:6787.746094,acc:0.000000 
[train]:step:28,loss:6685.555664,acc:0.000000 
[train]:step:29,loss:6585.392090,acc:0.000000 
[train]:step:30,loss:6678.966309,acc:0.000000 
[vaild]:step:30,loss:6871.312500,acc:0.000000 
[train]:step:31,loss:6750.029785,acc:0.000000 
[train]:step:32,loss:6762.289551,acc:0.000000 
[train]:step:33,loss:6762.516113,acc:0.000000 
[train]:step:34,loss:6661.260742,acc:0.000000 
[train]:step:35,loss:6746.633789,acc:0.000000 
[train]:step:36,loss:6698.163574,acc:0.000000 
[train]:step:

[train]:step:175,loss:6809.230469,acc:0.000000 
[train]:step:176,loss:6805.162598,acc:0.000000 
[train]:step:177,loss:6690.841309,acc:0.000000 
[train]:step:178,loss:6620.423828,acc:0.000000 
[train]:step:179,loss:6614.733887,acc:0.000000 
[train]:step:180,loss:6609.037109,acc:0.000000 
[vaild]:step:180,loss:6466.924316,acc:0.000000 
[train]:step:181,loss:6620.477051,acc:0.000000 
[train]:step:182,loss:6704.038574,acc:0.000000 
[train]:step:183,loss:6798.423340,acc:0.000000 
[train]:step:184,loss:6688.832520,acc:0.000000 
[train]:step:185,loss:6732.598145,acc:0.000000 
[train]:step:186,loss:6696.465820,acc:0.000000 
[train]:step:187,loss:6877.761230,acc:0.000000 
[train]:step:188,loss:6675.873535,acc:0.000000 
[train]:step:189,loss:6735.023926,acc:0.000000 
[train]:step:190,loss:6748.401855,acc:0.000000 
[vaild]:step:190,loss:7114.794434,acc:0.000000 
[train]:step:191,loss:6658.560547,acc:0.000000 
[train]:step:192,loss:6767.740723,acc:0.000000 
[train]:step:193,loss:6588.456055,acc:0.

[vaild]:step:330,loss:6733.189453,acc:0.000000 
[train]:step:331,loss:6767.589844,acc:0.000000 
[train]:step:332,loss:6718.222656,acc:0.000000 
[train]:step:333,loss:6564.606934,acc:0.000000 
[train]:step:334,loss:6698.502930,acc:0.000000 
[train]:step:335,loss:6724.657715,acc:0.000000 
[train]:step:336,loss:6693.106445,acc:0.000000 
[train]:step:337,loss:6879.773926,acc:0.000000 
[train]:step:338,loss:6714.915039,acc:0.000000 
[train]:step:339,loss:6718.493164,acc:0.000000 
[train]:step:340,loss:6731.926758,acc:0.000000 
[vaild]:step:340,loss:6854.973633,acc:0.000000 
[train]:step:341,loss:6730.921387,acc:0.000000 
[train]:step:342,loss:6720.316406,acc:0.000000 
[train]:step:343,loss:6705.214844,acc:0.000000 
[train]:step:344,loss:6696.243652,acc:0.000000 
[train]:step:345,loss:6593.520020,acc:0.000000 
[train]:step:346,loss:6615.464355,acc:0.000000 
[train]:step:347,loss:6777.386230,acc:0.010000 
[train]:step:348,loss:6521.984375,acc:0.000000 
[train]:step:349,loss:6857.241211,acc:0.

[train]:step:486,loss:6836.688965,acc:0.000000 
[train]:step:487,loss:6832.850098,acc:0.000000 
[train]:step:488,loss:6775.202637,acc:0.000000 
[train]:step:489,loss:6764.574219,acc:0.000000 
[train]:step:490,loss:6630.829590,acc:0.000000 
[vaild]:step:490,loss:6468.269043,acc:0.000000 
[train]:step:491,loss:6902.495605,acc:0.000000 
[train]:step:492,loss:6828.623535,acc:0.000000 
[train]:step:493,loss:6805.800781,acc:0.010000 
[train]:step:494,loss:6659.970215,acc:0.000000 
[train]:step:495,loss:6672.785156,acc:0.000000 
[train]:step:496,loss:6826.250000,acc:0.000000 
[train]:step:497,loss:6629.796875,acc:0.000000 
[train]:step:498,loss:6827.892090,acc:0.000000 
[train]:step:499,loss:6611.465820,acc:0.000000 
[train]:step:500,loss:6790.372559,acc:0.000000 
[vaild]:step:500,loss:7139.785645,acc:0.000000 
[train]:step:501,loss:6732.848633,acc:0.000000 
[train]:step:502,loss:6718.850586,acc:0.000000 
[train]:step:503,loss:6729.469238,acc:0.000000 
[train]:step:504,loss:6746.062500,acc:0.

...

[train]:step:12353,loss:6676.906250,acc:0.000000 
[train]:step:12354,loss:6665.377930,acc:0.000000 
[train]:step:12355,loss:6789.627441,acc:0.000000 
[train]:step:12356,loss:6672.635742,acc:0.000000 
[train]:step:12357,loss:6788.103516,acc:0.000000 
[train]:step:12358,loss:6616.643555,acc:0.000000 
[train]:step:12359,loss:6403.988281,acc:0.000000 
[train]:step:12360,loss:6874.259766,acc:0.000000 
[vaild]:step:12360,loss:6576.286133,acc:0.000000 
[train]:step:12361,loss:6660.705566,acc:0.000000 
[train]:step:12362,loss:6665.308594,acc:0.000000 
[train]:step:12363,loss:6578.188965,acc:0.000000 
[train]:step:12364,loss:6755.321289,acc:0.000000 
[train]:step:12365,loss:6500.473145,acc:0.000000 
[train]:step:12366,loss:6624.476074,acc:0.000000 
[train]:step:12367,loss:6637.920410,acc:0.000000 
[train]:step:12368,loss:6714.554199,acc:0.000000 
[train]:step:12369,loss:6526.036133,acc:0.000000 
[train]:step:12370,loss:6868.385742,acc:0.000000 
[vaild]:step:12370,loss:6822.169922,acc:0.000000 


[train]:step:12502,loss:6870.722656,acc:0.000000 
[train]:step:12503,loss:6764.825684,acc:0.000000 
[train]:step:12504,loss:6587.466797,acc:0.000000 
[train]:step:12505,loss:6639.079590,acc:0.000000 
[train]:step:12506,loss:6712.761719,acc:0.000000 
[train]:step:12507,loss:6650.265137,acc:0.000000 
[train]:step:12508,loss:6639.903809,acc:0.000000 
[train]:step:12509,loss:6662.070801,acc:0.000000 
[train]:step:12510,loss:6655.434570,acc:0.000000 
[vaild]:step:12510,loss:6685.670898,acc:0.000000 
[train]:step:12511,loss:6706.834473,acc:0.000000 
[train]:step:12512,loss:6791.618164,acc:0.000000 
[train]:step:12513,loss:6852.393555,acc:0.000000 
[train]:step:12514,loss:6568.342285,acc:0.000000 
[train]:step:12515,loss:6664.296875,acc:0.000000 
[train]:step:12516,loss:6826.826172,acc:0.000000 
[train]:step:12517,loss:6595.743652,acc:0.000000 
[train]:step:12518,loss:6757.131348,acc:0.000000 
[train]:step:12519,loss:6783.672363,acc:0.000000 
[train]:step:12520,loss:6677.890137,acc:0.010000 


[train]:step:12651,loss:6759.538086,acc:0.000000 
[train]:step:12652,loss:6819.443848,acc:0.000000 
[train]:step:12653,loss:6801.564453,acc:0.000000 
[train]:step:12654,loss:6669.178223,acc:0.000000 
[train]:step:12655,loss:6786.666016,acc:0.000000 
[train]:step:12656,loss:6606.060547,acc:0.000000 
[train]:step:12657,loss:6836.658691,acc:0.000000 
[train]:step:12658,loss:6680.648926,acc:0.000000 
[train]:step:12659,loss:6782.483887,acc:0.000000 
[train]:step:12660,loss:6704.285645,acc:0.010000 
[vaild]:step:12660,loss:7022.374023,acc:0.000000 
[train]:step:12661,loss:6931.610840,acc:0.000000 
[train]:step:12662,loss:6757.728516,acc:0.000000 
[train]:step:12663,loss:6707.387695,acc:0.000000 
[train]:step:12664,loss:6655.477539,acc:0.000000 
[train]:step:12665,loss:6697.218262,acc:0.000000 
[train]:step:12666,loss:6678.165039,acc:0.000000 
[train]:step:12667,loss:6772.975098,acc:0.000000 
[train]:step:12668,loss:6679.001953,acc:0.010000 
[train]:step:12669,loss:6633.662109,acc:0.000000 


[vaild]:step:12800,loss:6800.279297,acc:0.000000 
[train]:step:12801,loss:6748.095703,acc:0.000000 
[train]:step:12802,loss:6731.689941,acc:0.000000 
[train]:step:12803,loss:6754.513672,acc:0.020000 
[train]:step:12804,loss:6608.866699,acc:0.010000 
[train]:step:12805,loss:6473.594238,acc:0.000000 
[train]:step:12806,loss:6714.998535,acc:0.000000 
[train]:step:12807,loss:6764.950195,acc:0.000000 
[train]:step:12808,loss:6698.804199,acc:0.000000 
[train]:step:12809,loss:6604.909180,acc:0.000000 
[train]:step:12810,loss:6822.986328,acc:0.000000 
[vaild]:step:12810,loss:6683.688477,acc:0.000000 
[train]:step:12811,loss:6721.336426,acc:0.000000 
[train]:step:12812,loss:6749.386719,acc:0.000000 
[train]:step:12813,loss:6697.409180,acc:0.000000 
[train]:step:12814,loss:6602.075195,acc:0.000000 
[train]:step:12815,loss:6788.659180,acc:0.000000 
[train]:step:12816,loss:6705.991211,acc:0.000000 
[train]:step:12817,loss:6776.397949,acc:0.000000 
[train]:step:12818,loss:6751.776855,acc:0.000000 


KeyboardInterrupt: 

In [ ]:
# Test
X_test_all,Y_test_all = test_dataset.next_batch(1)
X_test,length = X_test_all
Y_test,length = Y_test_all
X_test = X_test[:,:,np.newaxis]
Y_test = Y_test[:,:,np.newaxis]
Y_hat = model.predict(X_test)
Y_hat = np.squeeze(Y_hat)
print(Y_hat)